In [1]:
from google.colab import files
import zipfile
import io
import os

# Step 1: Ask user to upload a ZIP file
print("Please upload your ZIP file:")
uploaded = files.upload()  # Opens file picker dialog

# Step 2: Extract the uploaded ZIP file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Unzipping {filename}...")
        with zipfile.ZipFile(io.BytesIO(uploaded[filename]), 'r') as zip_ref:
            zip_ref.extractall('/content/')
        print("Extraction complete!")
        print("Files extracted to /content/")

# Step 3: List extracted files
print("\nExtracted folder structure:")
!ls -R /content/

Please upload your ZIP file:


Saving nas-ga-basic-main.zip to nas-ga-basic-main.zip
Unzipping nas-ga-basic-main.zip...
Extraction complete!
Files extracted to /content/

Extracted folder structure:
/content/:
nas-ga-basic-main  nas-ga-basic-main.zip  sample_data

/content/nas-ga-basic-main:
LICENSE  model_cnn.py  model_ga.py  nas_run.py	README.md

/content/sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [4]:
# ---------------------------
# Run both: Tournament vs Roulette
# ---------------------------
import os, shutil, zipfile, pickle, time
from pathlib import Path

ZIP_PATH = "/mnt/data/nas-ga-basic-main.zip"   # your uploaded zip (from conversation history)
CONTENT = "/content"
PROJECT_DIR = None

# 0) Unzip if project folder not present
if not any(p for p in os.listdir(CONTENT) if 'nas' in p.lower() and os.path.isdir(os.path.join(CONTENT, p))):
    print("Project folder not found in /content. Unzipping uploaded ZIP...")
    if not os.path.exists(ZIP_PATH):
        raise FileNotFoundError(f"ZIP file not found at {ZIP_PATH}. Upload first or update ZIP_PATH.")
    shutil.copy(ZIP_PATH, CONTENT)
    zip_target = os.path.join(CONTENT, os.path.basename(ZIP_PATH))
    with zipfile.ZipFile(zip_target, 'r') as zf:
        zf.extractall(CONTENT)
    print("Unzipped to /content/")
else:
    print("Project folder found in /content/ - skipping unzip.")

# 1) Detect project folder (prefer one with 'nas' in name)
candidates = [p for p in os.listdir(CONTENT) if os.path.isdir(os.path.join(CONTENT, p))]
for c in candidates:
    if 'nas' in c.lower():
        PROJECT_DIR = os.path.join(CONTENT, c)
        break
if PROJECT_DIR is None:
    # fallback to most recently modified dir in /content
    dirs = [(os.path.join(CONTENT, d), os.path.getmtime(os.path.join(CONTENT, d))) for d in candidates]
    dirs.sort(key=lambda x: x[1], reverse=True)
    PROJECT_DIR = dirs[0][0] if dirs else CONTENT

print("Using project dir:", PROJECT_DIR)

# 2) Add project dir to sys.path and cd into it
import sys
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)
os.chdir(PROJECT_DIR)
print("Working directory:", os.getcwd())

# 3) Ensure outputs base exists
run_base = os.path.join(PROJECT_DIR, 'outputs')
os.makedirs(run_base, exist_ok=True)
# determine next two run ids
existing_runs = [d for d in os.listdir(run_base) if d.startswith('run_') and os.path.isdir(os.path.join(run_base, d))]
next_run = len(existing_runs) + 1
run_tournament = next_run
run_roulette = next_run + 1

# 4) Import project modules (model_ga, model_cnn). If import fails, raise informative error.
try:
    from model_ga import GeneticAlgorithm
    from model_cnn import CNN
except Exception as e:
    raise ImportError(f"Failed to import project modules (model_ga/model_cnn). Error: {e}")

# 5) Prepare dataset loaders (small subsets for quick runs)
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
data_root = os.path.join(PROJECT_DIR, 'data')
os.makedirs(data_root, exist_ok=True)

trainset = CIFAR10(root=data_root, train=True, download=True, transform=transform)
valset = CIFAR10(root=data_root, train=False, download=True, transform=transform)
# small subsets to keep runtime short (adjust if you want longer runs)
train_subset = Subset(trainset, range(min(5000, len(trainset))))
val_subset   = Subset(valset,   range(min(1000, len(valset))))

train_loader = DataLoader(train_subset, batch_size=256, shuffle=True)
val_loader   = DataLoader(val_subset, batch_size=256, shuffle=False)

# helper logging
def log(msg):
    print(msg, flush=True)

# 6) Function to run an experiment (selection_method: 'tournament' or 'roulette')
def run_experiment(selection_method, population_size=5, generations=2, run_id=1):
    log("\n" + "="*60)
    log(f"STARTING RUN {run_id}  (selection={selection_method})")
    log("="*60)
    ga = GeneticAlgorithm(
        population_size=population_size,
        generations=generations,
        mutation_rate=0.3,
        crossover_rate=0.7
    )
    t0 = time.time()
    best_arch = ga.evolve(train_loader, val_loader, device, run=run_id, selection_method=selection_method)
    elapsed = time.time() - t0
    # create run dir (evolve may have created it; ensure exists)
    run_dir = os.path.join(run_base, f'run_{run_id}')
    os.makedirs(run_dir, exist_ok=True)
    # save best arch
    with open(os.path.join(run_dir, "best_arch.pkl"), 'wb') as f:
        pickle.dump(best_arch, f)
    log(f"Completed run_{run_id} in {elapsed:.1f} s. Best arch saved to {run_dir}/best_arch.pkl")
    # return summary
    summary = {
        'run_id': run_id,
        'selection': selection_method,
        'genes': getattr(best_arch, 'genes', None),
        'accuracy': getattr(best_arch, 'accuracy', None),
        'fitness_original': getattr(best_arch, 'fitness_original', getattr(best_arch, 'fitness', None)),
        'fitness_weighted': getattr(best_arch, 'fitness_weighted', None),
        'param_count': sum(p.numel() for p in CNN(getattr(best_arch,'genes',{})).parameters()) if getattr(best_arch,'genes',None) else None,
        'run_dir': run_dir
    }
    return summary

# 7) Run both experiments back-to-back
summary_tourn = run_experiment('tournament', population_size=5, generations=2, run_id=run_tournament)
summary_roul  = run_experiment('roulette',   population_size=5, generations=2, run_id=run_roulette)

# 8) Compare results
log("\n" + "="*60)
log("COMPARISON SUMMARY")
log("="*60)
log(f"Run {summary_tourn['run_id']} (Tournament):")
log(f"  Accuracy: {summary_tourn['accuracy']}")
log(f"  Fitness (orig): {summary_tourn['fitness_original']}")
log(f"  Fitness (weighted): {summary_tourn['fitness_weighted']}")
log(f"  Params: {summary_tourn['param_count']}")
log(f"  Saved at: {summary_tourn['run_dir']}\n")

log(f"Run {summary_roul['run_id']} (Roulette):")
log(f"  Accuracy: {summary_roul['accuracy']}")
log(f"  Fitness (orig): {summary_roul['fitness_original']}")
log(f"  Fitness (weighted): {summary_roul['fitness_weighted']}")
log(f"  Params: {summary_roul['param_count']}")
log(f"  Saved at: {summary_roul['run_dir']}\n")

# 9) Quick winner decision (by original fitness)
winner = summary_tourn if (summary_tourn['fitness_original'] or 0) >= (summary_roul['fitness_original'] or 0) else summary_roul
log("WINNER (by original fitness):")
log(f"  Run {winner['run_id']} - selection: {winner['selection']}")
log(f"  Accuracy: {winner['accuracy']}, Fitness (orig): {winner['fitness_original']}, Params: {winner['param_count']}")
log("\nAll done. Two runs completed and saved to outputs directory.")


Project folder found in /content/ - skipping unzip.
Using project dir: /content/nas-ga-basic-main
Working directory: /content/nas-ga-basic-main
Using device: cuda

STARTING RUN 4  (selection=tournament)
Starting with 5 Population:
[Arch(conv=2, acc=0.0000, fit=0.0000), Arch(conv=4, acc=0.0000, fit=0.0000), Arch(conv=4, acc=0.0000, fit=0.0000), Arch(conv=2, acc=0.0000, fit=0.0000), Arch(conv=4, acc=0.0000, fit=0.0000)]


Generation 1/2
Evaluating architecture 1/5... Fitness (orig): 0.5398, Accuracy: 0.5840
Weighted fitness: 0.5840
Evaluating architecture 2/5... Fitness (orig): 0.6530, Accuracy: 0.6770
Weighted fitness: 0.6770
Evaluating architecture 3/5... Fitness (orig): 0.5178, Accuracy: 0.5210
Weighted fitness: 0.5210
Evaluating architecture 4/5... Fitness (orig): 0.5319, Accuracy: 0.5740
Weighted fitness: 0.5740
Evaluating architecture 5/5... Fitness (orig): 0.6252, Accuracy: 0.6370
Weighted fitness: 0.6370

Sorting population in terms of fitness score (high -> low) ...
Best in gene

In [8]:
import shutil
from google.colab import files

# Path to your folder
folder_path = '/content/nas-ga-basic-main'

# Create ZIP file
zip_path = '/content/nas-ga-basic-main.zip'
shutil.make_archive('/content/nas-ga-basic-main', 'zip', folder_path)

# Download to local computer
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>